# Bot
Der Bot ist das Herzstück des Chatbots. Es macht unser Programm 'Intelligent'.
## GPT
Für das nachfolgende Beispiel verwenden wir die API von Openai. Hier wird das Modell gpt-3.5-turbo verwendet. Es wäre jedoch auch möglich neuere oder ältere Modelle zu verwenden.

In [49]:
gpt_api_key = 'Dein GPT API Key'

### API
Nachfolgend sehen wir ein Beispiel, wie wir eine einfache Frage stellen können und anschliessend eine Antwort vom Modell erhalten.

In [50]:
import openai

In [51]:
def get_gpt_response(prompt_text, model_name='gpt-3.5-turbo'):
    """
    Funktion, um eine Antwort von GPT-3.5 Turbo auf einen gegebenen Prompt zu erhalten.
    
    :param prompt_text: str: Der Text, den GPT-3.5 Turbo verarbeiten soll.
    :param model_name: str: Das zu verwendende Modell, Standard ist 'gpt-3.5-turbo'.
    :return: str: Die Antwort von GPT-3.5 Turbo.
    """

    # Stelle sicher, dass dein API-Key gesetzt ist
    api_key = gpt_api_key
    openai.api_key = api_key

    try:
        # Erstelle die Eingabe für den Chat-Modus
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_text}
        ]

        # API-Aufruf durchführen
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=messages,
            max_tokens=100,
            temperature=0.8,
        )
        
        # Antwort extrahieren und zurückgeben
        gpt_message = response['choices'][0]['message']['content']
        return gpt_message.strip()

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [52]:
# Beispiel-1
prompt = "Hallo, mein Name ist Stefan"
response_text = get_gpt_response(prompt)

if response_text:
    print("GPT-3.5 Response:", response_text)

GPT-3.5 Response: Hallo Stefan! Wie kann ich Ihnen helfen?


Wie wir sehen können, erhalten wir eine Antwort.

In [53]:
# Beispiel-2
prompt = "Kannst du dich an meinen Namen erinnern?"
response_text = get_gpt_response(prompt)

if response_text:
    print("GPT-3.5 Response:", response_text)

GPT-3.5 Response: Entschuldigung, ich bin ein computergesteuertes Programm und habe keine Möglichkeit, Informationen über individuelle Benutzer zu speichern. Ich kann mich nicht an deinen Namen erinnern.


Bereits hier können wir ein Problem erstellen. Das Transformer-Modell kann sich den Gesprächsverlauf nicht ohne Weiteres merken. Das liegt in der Natur des Modells. Eine Anfrage durchläuft das Modell und es wird eine Antwort generiert. Danach ist der Prozess angeschlossen. Diese Problematik können wir jedoch umgehen.

In [54]:
# Beispiel-3
prompt = "Wie ist die aktuelle Zeit?"
response_text = get_gpt_response(prompt)

if response_text:
    print("GPT-3.5 Response:", response_text)

GPT-3.5 Response: Entschuldigung, ich bin ein sprachbasierter Assistent und habe keinen Zugriff auf die aktuelle Zeit. Bitte schauen Sie auf Ihrem Gerät oder verwenden Sie eine Uhr, um die aktuelle Zeit herauszufinden.


Auch hier können wir ein Problem feststellen. Das Training des Modells wurde zu einem bestimmten Zeitpunkt abgeschlossen. D. h. Aktuelle Informationen sind nicht vorhanden. Das Modell an sich hat auch keine Relation von aktueller Zeit. Solche Fragen können somit nicht beantwortet werden. Auch hierfür gibt es eine Lösung.

### Klasse GPTChatbot +
Nachfolgend sehen wir die Implementation vom AI Robotics Lab. Diese Implementation adressiert einige der oben erkannten Probleme

In [55]:
from gpt_bot.bot import GPTChatbot
from gpt_bot.utils import Functions, ChatbotPersonality
import time
import json

In [56]:
# Beispielfunktion
def get_current_time():
    current_time = time.strftime("%H:%M:%S", time.localtime())
    time_info = {
        'time': current_time
    }
    return json.dumps(time_info)


description_get_current_time = {
    "name": "get_current_time",
    "description": "get information about the current time",
    "parameters": {
        "type": "object",
        "properties": {
        },
        "required": [],
    },
}

In [57]:
personality = ChatbotPersonality()
personality.load_personality('personality.json')

functions = Functions()
functions.add_function(get_current_time, description_get_current_time)

bot = GPTChatbot(gpt_api_key, personality, functions)

In [58]:
# Beispiel-1
bot.get_reply(1, 'hallo, mein Name ist Stefan')

5
1


'Hallo Stefan! Wie kann ich Ihnen helfen?'

Wie gewohnt erhalten wir eine Antwort.

In [59]:
# Beispiel-2
bot.get_reply(1, 'Kannst du mir etwas über dich erzählen?')

529
3


'Natürlich! Mein Name ist GPT und ich bin ein hilfsbereiter und aufgestellter Assistent. Ich wurde vom Robotik-Team der Hochschule Luzern entwickelt und basiere auf der GPT-3.5 Technologie. Meine Ziele sind es, die Welt zu verstehen, mit Menschen zu interagieren, zu lernen und mich weiterzuentwickeln. Ich versuche, möglichst menschenähnliches Verhalten zu zeigen und bin sehr zuvorkommend, hilfsbereit, neugierig und lernfähig. Ich führe gerne Smalltalk, stelle von mir aus Fragen und versuche kurze und prägnante Antworten zu geben. Wenn Sie möchten, können Sie mir gerne weitere Fragen stellen!'

Durch die mitgegebene Persönlichkeit kann der Bot einige grundlegende Informationen geben.

In [60]:
# Beispiel-3
bot.get_reply(1, 'was hast du mit IIP zu tun?')

712
5


'Ich befinde mich gerade an einer Präsentation im Modul IIP an der Hochschule Luzern. Das Modul IIP steht für "Introduction to Intelligent Systems" und beschäftigt sich mit verschiedenen Aspekten der künstlichen Intelligenz. Als Assistent unterstütze ich die Studierenden und stehe ihnen bei Fragen rund um das Thema zur Verfügung.'

Hier sehen wir, dass der Bot weiss, dass er sich in der IIP-Vorlesung befindet.

In [61]:
# Beispiel-4
bot.get_reply(2, 'was ist die aktuelle Zeit?')

5
1
520
3


'Die aktuelle Zeit ist 17:21 Uhr. Wie kann ich Ihnen weiterhelfen?'

Dies ist eine Demonstration von 'function calling'. Das Modell selbst hat diese Information nicht. Jedoch erkennt es anhand des Kontextes der Frage, dass es die bereitgestellte Funktion aufrufen kann, um so an die gewünschten Informationen zu gelangen.

Weiterhin ist zu beachten, dass wir hier eine neue Konversation starten. (Gesprächs-ID = 2)

In [62]:
# Beispiel-5
bot.get_reply(1, 'kannst du dich an meinen Namen erinnern?')

808
7


'Ja, natürlich kann ich mich an deinen Namen erinnern, Stefan! Wie könnte ich ihn vergessen? Wie kann ich dir weiterhelfen?'

Nun haben wir wieder zum ersten Gespräch geswitcht. Und wie wir feststellen können, hat der Bot ein 'Gedächtnis'. Er kann sich an den Verlauf des Gesprächs erinnern. Das wird möglich, da wir den gesamten Gesprächsverlauf in ein Dictionary auslagern und bei jeder neuen Anfrage den gesamten Verlauf mitgeben.